In [169]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
plt.style.use("seaborn")
np.random.seed(42)

In [168]:
class Node:
    def __init__(self, id, value):
        self.id = id
        self.value = value

    def __str__(self) -> str:
        return str(self.value)
    
    def __repr__(self) -> str:
        return f"{self.id}/{self.value}"

def edge_cost(frag1:str, frag2:str):
    if frag1 == frag2:
        return 0
    cost = len(frag1)
    for i in range(len(frag1)):
        sub1 = frag1[i+1:]
        sub2 = frag2[0:-i-1]
        # print(sub1, sub2)
        if sub1 == sub2:
            return i+1
    return cost

def create_cost_matrix(nodes):
    matrix = np.zeros([len(nodes), len(nodes)])
    for frag_from in nodes:
        for frag_to in nodes:
            cost = edge_cost(frag_from.value, frag_to.value)
            matrix[frag_from.id][frag_to.id] = cost
    return matrix

In [167]:
import random
def sequence_length(sequence:list[Node]):
    if len(sequence) == 0:
        return 0
    length = len(sequence[0].value)
    for node1, node2 in zip(sequence, sequence[1:]):
        length += cost_matrix[node1.id][node2.id]
    return length

def random_good_path(nodes, initial_sequence = None):
    nodes_local = nodes.copy()
    
    if initial_sequence is None:
        sequence = []
        current = random.choice(nodes_local)
        sequence.append(current)
        nodes_local.remove(current)
        current_seq_len = len(current.value)
    else:
        sequence = initial_sequence
        current = sequence[-1]
        current_seq_len = sequence_length(sequence)

    can_continue = True
    while can_continue:
        avialble_ids = [x.id for x in nodes_local]
        if len(avialble_ids) == 0:
            can_continue = False
            break
        
        min_distance = min(cost_matrix[current.id][avialble_ids])
        closest_nodes = [ x for x in nodes_local if cost_matrix[current.id][x.id]==min_distance ]
        next_node = random.choice(closest_nodes)
        
        current_seq_len +=  cost_matrix[current.id][next_node.id]
        if current_seq_len > max_seq_len or min_distance == fragment_len:
            can_continue = False
            break
        else:
            sequence.append(next_node)
            nodes_local.remove(next_node)
            current = next_node
               
    return sequence

    
def generate_initial_pop(nodes, population_num):
    population = []

    for i in range(population_num):
        population.append(random_good_path(nodes))
    return population


def print_sequence(sequence):
    print(len(sequence), sequence_length(sequence), sequence)

# initial_population = generate_initial_pop(nodes, population_size)
# for sequence in initial_population:
#     print_sequence(sequence)

In [166]:
from numpy.random import choice

def f_score(sequence)->float:
        no_nodes = len(sequence) / max_nodes
        length = sequence_length(sequence) / max_seq_len
        # gives more weight to no_nodes
        score = 1.01 * (no_nodes * length) / (0.01*no_nodes + length)
        return score

def calculate_propabilities(population):
    scores = np.asarray(
        [f_score(x) for x in population]
    )
    seq_propabilities = scores / np.sum(scores)
    return seq_propabilities
    
propabilities = calculate_propabilities(initial_population)

def select(population, selectivity, propabilities):
    selected_num = int(len(population) * selectivity)
    draw = choice(population, selected_num, p=propabilities).tolist()
    return draw

def get_best_sequence(population:list[list[Node]]):
    scores = [(f_score(x),x) for x in population]
    scores_sorted = sorted(scores, key=lambda x : x[0], reverse=True)
    best_score, best_seq = scores_sorted[0]
    return best_seq
    

# natural_selection = select(initial_population, selectivity, propabilities)
# print(propabilities)
# for sequence in natural_selection:
#     print_sequence(sequence)


In [165]:
def crossover(path1:list[Node], path2:list[Node]):
    geneA_idx = random.randint(0, len(path1))
    geneB_idx = random.randint(0, len(path1))
    
    startGene = min(geneA_idx, geneB_idx)
    endGene = max(geneA_idx, geneB_idx)

    if startGene == endGene:
        endGene += 1

    child = [path1[i] for i in range(startGene, endGene)]
    child_len = sequence_length(child)

    for next_node in path2:
        if next_node in child:
            continue

        child_len += cost_matrix[child[-1].id][next_node.id]
        if child_len <= max_seq_len:
            child.append(next_node)
        else:
            break

    child_len = sequence_length(child)
    if child_len < max_seq_len:
        nodes_left = [item for item in nodes if item not in child]
        child = random_good_path(nodes_left, initial_sequence=child)

    return child



def swap_nodes(sequence:list[Node], mutation_rate:float):
    sequence_copy = sequence.copy()
    for swapped in range(len(sequence_copy)):
        swap_with = int(random.random() * len(sequence))

        sequence_copy[swapped], sequence_copy[swap_with] = sequence_copy[swap_with], sequence_copy[swapped]
        if not (sequence_length(sequence_copy) <= max_seq_len and random.random() < mutation_rate):
            # reverse swap
            sequence_copy[swapped], sequence_copy[swap_with] = sequence_copy[swap_with], sequence_copy[swapped]
    return sequence_copy
        


def insert_nodes(sequence:list[Node], mutation_rate:float):
    sequence_copy = sequence.copy()
    nodes_left = [item for item in nodes if item not in sequence_copy]

    for insert_idx in range(len(sequence_copy)):

        insert_node = random.choice(nodes_left)
        sequence_copy.insert(insert_idx, insert_node)

        if not(sequence_length(sequence_copy) <= max_seq_len and random.random() < mutation_rate):
            sequence_copy.remove(insert_node)
    return sequence_copy

def breed_population(parents: list[Node], population_size, crossover_p):
    children = []
    for _ in range(population_size):
        if np.random.rand() < crossover_p:
            parent1, parent2 = random.sample(parents, k=2)
            child = crossover(parent1, parent2)
        else:
            child = random.choice(parents)
        children.append(child)
    return children

    
def mutate_population(children: list[Node], mutate_p):
    mutated = []
    for child in children:
        mutated_child = swap_nodes(child, mutate_p)
        mutated_child = insert_nodes(mutated_child, mutate_p)
        mutated.append(mutated_child)
    return mutated


# children = breed_population(natural_selection, population_size, 0.5)
# mutated = mutate_population(children, 0.1)
# for sequence in natural_selection:
#     print_sequence(sequence)

# print("Breeded and mutated")
# for sequence in mutated:
#     print_sequence(sequence)

In [170]:

input_file = "9.200-40.txt"
fragment_len = 10 
max_seq_len = 209 # changes with different files

population_size = 100
selectivity = 0.1
crossover_p = 0.5
mutate_p = 0.1
iterations = 10

fragments = []
with open(input_file) as f:
    fragments = f.read().splitlines()

nodes =[ Node(id, frag) for id,frag in enumerate(fragments)]
cost_matrix = create_cost_matrix(nodes)
max_nodes = len(nodes)

history = []
best_sequence = None

population = generate_initial_pop(nodes, population_size)
best_sequence = get_best_sequence(population)
history.append(best_sequence)
for _ in range(iterations):
    propabilities = calculate_propabilities(population)
    parents = select(population, selectivity, propabilities)
    children = breed_population(parents, population_size, crossover_p)
    mutated_children = mutate_population(children, mutate_p)
    population = mutated_children
    best_sequence = get_best_sequence(population)
    history.append(best_sequence)

    
for seq in history:
    print_sequence(seq)

C:\Users\motme\AppData\Local\Temp\ipykernel_33724\3272593274.py:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  draw = choice(population, selected_num, p=propabilities).tolist()


158 205.0 [96/GCCGATCCGC, 44/CCGATCCGCC, 56/CGATCCGCCG, 86/GATCCGCCGC, 12/ATCCGCCGCC, 138/TCCGCCGCCC, 46/CCGCCGCCCG, 60/CGCCCGTCCA, 95/GCCCGTCCAC, 42/CCCGTCCACA, 50/CCGTCCACAC, 126/GTCCACACCC, 135/TCCACACCCG, 28/CCACACCCGC, 19/CACACCCGCC, 1/ACACCCGCCG, 20/CACCCGCCGC, 4/ACCCGCCGCC, 40/CCCGCCGCCA, 45/CCGCCGCCAG, 61/CGCCGCCAGC, 97/GCCGCCAGCT, 58/CGCCAGCTCA, 93/GCCAGCTCAC, 29/CCAGCTCACC, 22/CAGCTCACCA, 7/AGCTCACCAT, 104/GCTCACCATG, 3/ACCATGGATG, 33/CCATGGATGA, 25/CATGGATGAT, 17/ATGGATGATG, 107/GGATGATGAT, 15/ATGATGATAT, 148/TGATGATATC, 87/GATGATATCG, 14/ATGATATCGC, 147/TGATATCGCC, 85/GATATCGCCG, 11/ATATCGCCGC, 13/ATCGCCGCGC, 142/TCGCCGCGCT, 62/CGCCGCGCTC, 47/CCGCGCTCGT, 63/CGCGCTCGTC, 101/GCGCTCGTCG, 65/CGCTCGTCGT, 106/GCTCGTCGTC, 79/CTCGTCGTCG, 144/TCGTCGTCGA, 72/CGTCGTCGAC, 71/CGTCGACAAC, 127/GTCGACAACG, 141/TCGACAACGG, 54/CGACAACGGC, 83/GACAACGGCT, 67/CGGCTCCGGC, 116/GGCTCCGGCA, 105/GCTCCGGCAT, 78/CTCCGGCATG, 139/TCCGGCATGT, 66/CGGCATGTGC, 110/GGCATGTGCA, 27/CATGTGCAAG, 156/TGTGCAAGGC, 